## 3.3 데이터 전처리

In [1]:
# 작업 경로 설정
import os
os.chdir(r"C:\dacon\ch03")

In [1]:
import pandas as pd

# 데이터 로드
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
bts= pd.read_csv('data/bus_bts.csv')
jeju_life = pd.read_csv('./jeju_financial_life_data.csv')
weather = pd.read_csv('./weather.csv', encoding='cp949')
rain = pd.read_csv('./rain.csv', encoding='utf-8')

In [2]:
# 시각화에 필요한 라이브러리 임포트
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 내부에 결과를 출력하도록 설정
%matplotlib inline
# 시각화 한글 폰트 설정
# 윈도우 사용자
# plt.rc('font', family='Malgun Gothic')
# Mac 사용자
plt.rc('font', family='AppleGothic')
# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False) 
# 분석에 문제가 없는 경고 메세지는 숨긴다.
import warnings
warnings.filterwarnings('ignore') 

In [10]:
# 학습 데이터와 테스트 데이터를 구분하기 위한 변수 생성
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터 통합
df = pd.concat([train, test], axis=0)

### 3.3.1 내부 데이터를 통한 변수 생성

#### 3.3.1.1 탐색적 데이터 분석을 통한 변수

* 요일을 나타내는 변수(weekday)

In [11]:
# datetime 변수형으로 변환
df['date'] = pd.to_datetime(df['date'])
# 요일 추출 (0-월요일~6-일요일)
df['weekday'] = df['date'].dt.weekday

In [12]:
df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


* 요일별 평균 탑승객 수

In [13]:
# 요일별 평균 탑승 승객수를 구하는 함수
def week_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 'weekdaymean'변수 생성
    df['weekdaymean'] = 1

    # 각 요일에 해당하는 인덱스 추출
    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 인덱스를 활용하여 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 승객수로 변경 
    df.iloc[index0,-1] = train_data.query('weekday==0')['18~20_ride'].mean()
    df.iloc[index1,-1] = train_data.query('weekday==1')['18~20_ride'].mean()
    df.iloc[index2,-1] = train_data.query('weekday==2')['18~20_ride'].mean()
    df.iloc[index3,-1] = train_data.query('weekday==3')['18~20_ride'].mean()
    df.iloc[index4,-1] = train_data.query('weekday==4')['18~20_ride'].mean()
    df.iloc[index5,-1] = train_data.query('weekday==5')['18~20_ride'].mean()
    df.iloc[index6,-1] = train_data.query('weekday==6')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = week_mean()

In [14]:
df[['weekdaymean']].head()

,weekdaymean
0,1.034282
1,1.034282
2,1.034282
3,1.034282
4,1.034282


* 버스 종류별 평균 탑승객 수

In [15]:
# 버스 종류별 평균 탑승 승객수를 구하는 함수
def inout_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 in_out_mean 변수 생성
    df['in_out_mean'] = 1

    # 버스 종류별 인덱스 추출
    in_index = df.query('in_out == "시내"').index # 시내버스
    out_index = df.query('in_out == "시외"').index # 시외버스

    # 인덱스를 활용하여 in_out_mean의 값을 각 버스 종류에 맞는 
    # 평균 탑승 승객수로 변경
    df.iloc[in_index,-1] = train_data.query('in_out == "시내"')['18~20_ride'].mean()
    df.iloc[out_index,-1] = train_data.query('in_out == "시외"')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = inout_mean()

In [16]:
df[['in_out_mean']].head()

,in_out_mean
0,2.044345
1,2.044345
2,2.044345
3,1.228499
4,1.228499


* 일별 오전 시간대의 총 탑승객 수

In [17]:
# 날짜별 오전시간에 탑승한 총 승객수
f=df.groupby('date')['6~7_ride','7~8_ride','8~9_ride','9~10_ride','10~11_ride'
                    ].sum().reset_index()

# 변수명 바꾸기
f.columns=['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
           '10~11_ride_sum']

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, f, how='left', on='date')

In [18]:
df[['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
    '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


#### 3.3.1.2 도메인 조사를 통한 변수

* 배차 간격

In [19]:
# 탑승 날짜를 datetime형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date','geton_station_code','bus_route_id',
                        'geton_time2'],ascending=True).reset_index()

In [20]:
# 반복문 실행 시간이 많이 소요되어 상태 진행률을 확인할 수 있는 tqdm 라이브러리 사용
from tqdm import notebook

# 시차를 넣기 위한 빈 리스트 생성
interval_lst = []
for i in notebook.tqdm(range(0,len(f)-1)):
    # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) & 
        (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) & 
        (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
        # 탑승 날짜의 차를 interval_lst에 넣기
        interval_lst.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)
    else:
       # 같지 않다면 0을 interval_lst에 넣기
        interval_lst.append(0)

interval_lst.insert(0, '0')

  0%|          | 0/2409413 [00:00<?, ?it/s]

In [21]:
f['interval'] = interval_lst # interval_lst를 'interval'변수로 생성 
# 'interval' 변수의 값을 문자형으로 변환 후 '시:분:초'의 형태로 변환
f['interval'] = f['interval'].astype(str).str[7:].replace('','00:00:00')
f['bus_route_id'] = f['bus_route_id'].astype(object)

In [22]:
# 분으로 변환하는 함수
def get_min(time_str):
    h, m, s = time_str.split(':')
    return (int(h) * 3600 + int(m) * 60 + int(s)) / 60
# 'interval' 함수의 모든 값에 대해 함수 적용하여 나온 값을 time리스트에 넣기
time = []
for i in f['interval'] :
    time.append(get_min(i))

f['time'] = time # time리스트를 'time' 변수로 생성
f['time'] = f['time'].astype(int) # 'time' 변수를 int 타입으로 변경
f = f[f['time'] > 3] # 간격이 3분보다 작은 것 제외 
f = f[f['time'] < 180] # 간격이 3시간보다 큰 것 제외

In [23]:
# 버스 노선별 버스 간격의 평균 'bus_interval’변수 생성
data = f.groupby('bus_route_id')['time'].agg([('bus_interval', 'mean')]
                                            ).reset_index()
data['bus_interval'] = data['bus_interval'].astype(int)

# 변수 생성에 많은 시간이 소요되므로 csv 파일로 저장
data.to_csv('bus_interval.csv', index = False)

In [24]:
# csv 파일이 성공적으로 저장됐다면 다음 과정부터 진행
import numpy as np
df['bus_route_id'] = df['bus_route_id'].astype(np.int64)
bus_interval = pd.read_csv("bus_interval.csv")

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, bus_interval, how = 'left', on = 'bus_route_id')
df['bus_interval'] = df['bus_interval'].fillna(9999) # 병합 중 생기는 NA값 처리

In [25]:
df[['bus_interval']].head()

,bus_interval
0,34.0
1,34.0
2,34.0
3,34.0
4,34.0


* 수요가 많을 것으로 예상되는 정류장

In [26]:
# school 변수 생성
# 정류장명에 '고등학교'가 포함되어 있는 행 뽑기 
highschool_station = df[df['station_name'].str.contains('고등학교')]
# '고등학교'가 포함되어 있는 정류장명을 중복 없이 리스트에 저장
highschool = list(highschool_station['station_name'].unique())
# 정류장명에 '대학교'가 포함되어 있는 행 뽑기
university_station = df[df['station_name'].str.contains('대학교')]
# '대학교'가 포함되어 있는 정류장명을 중복 없이 리스트에 저장
university = list(university_station['station_name'].unique())

# '고등학교'와 '대학교' 근처 정류장 구분 함수
def schoolstation(x):
    # highschool 리스트에 정류장명이 포함되어 있다면 1을 반환
    if x in highschool:
        return 1
    # university 리스트에 정류장명이 포함되어 있다면 1을 반환
    elif x in university:
        return 1
    # 둘 다 포함되지 않는 정류장명이라면 0을 반환
    else:
        return 0

# 모든 정류장명에 대해 적용
df['school'] = df['station_name'].apply(schoolstation) 

In [27]:
df[['school']].head()

,school
0,0
1,0
2,0
3,0
4,0


In [28]:
# school 변수와 동일한 방법으로 transfer 변수 생성
transfer_station = df[df['station_name'].str.contains('환승')]
transfers = list(transfer_station['station_name'].unique())

airport_station = df[df['station_name'].str.contains('공항')]
airport = list(airport_station['station_name'].unique())

terminal_station = df[df['station_name'].str.contains('터미널')]
terminal = list(terminal_station['station_name'].unique())

# 항상 수요가 많은 정류장 구분 함수
def transfer_station(x):
    if x in transfers:
        return 1
    elif x in airport:
        return 1
    elif x in terminal:
        return 1
    else:
        return 0

df['transfer'] = df['station_name'].apply(transfer_station)

In [29]:
df[['transfer']].head()

,transfer
0,0
1,0
2,0
3,1
4,0


* 연휴

In [30]:
# 날짜에서 연휴 구분 함수
def holiday(x):
    # 공휴일이라면 1을 반환
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        # 공휴일이 아니라면 0을 반환
        return 0

    # 모든 날짜에 적용
df['holiday'] = df['date'].apply(holiday)
df[['holiday']].head()

,holiday
0,0
1,0
2,0
3,0
4,0


#### 3.3.1.3. 시간대를 활용한 변수

* 승하차 시간대 통합 변수(t~t+2)

In [31]:
# 오전 시간대에 2시간 동안 승차한 인원수의 합계
df['68a'] = df['6~7_ride'] + df['7~8_ride'] 
df['810a'] = df['8~9_ride'] + df['9~10_ride']
df['1012a'] = df['10~11_ride'] + df['11~12_ride']

# 오전 시간대에 2시간 동안 하차한 인원수의 합계
df['68b'] = df['6~7_takeoff'] + df['7~8_takeoff'] 
df['810b'] = df['8~9_takeoff'] + df['9~10_takeoff']
df['1012b'] = df['10~11_takeoff'] + df['11~12_takeoff']

In [32]:
df[['68a', '810a', '1012a', '68b', '810b', '1012b']].head()

,68a,810a,1012a,68b,810b,1012b
0,1.0,7.0,8.0,0.0,0.0,0.0
1,5.0,6.0,11.0,0.0,0.0,0.0
2,2.0,2.0,0.0,0.0,0.0,0.0
3,17.0,32.0,30.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0


* 오전 시간의 승객 수

In [33]:
# 오전 시간(6시~12시)에 승차한 승객 수의 합계
df['ride_sum'] = df[['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', 
                     '10~11_ride', '11~12_ride']].sum(axis=1)

# 오전 시간(6시~12시)에 하차한 승객 수의 합계
df['takeoff_sum'] = df[['6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff',
                        '10~11_takeoff', '11~12_takeoff']].sum(axis=1)

In [34]:
df[['ride_sum', 'takeoff_sum']].head()

,ride_sum,takeoff_sum
0,16.0,0.0
1,22.0,0.0
2,4.0,0.0
3,79.0,0.0
4,0.0,1.0


In [35]:
# 'bus_route_id'와 'station_code'를 결합해 route_station 생성
df['route_station'] = df['bus_route_id'].astype(str) + ',' \
                    + df['station_code'].astype(str)

# 버스노선별 오전시간의 총 승객수와 평균 승객수 구하기
data = df.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()

# 변수명을 route_station, 1012a_mean, 1012a_sum으로 바꾸기
data.rename(columns = {'mean':'1012a_mean', 'sum':'1012a_sum'}, inplace=True)

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, data, how='left', on='route_station')

In [36]:
df[['1012a_mean','1012a_sum']].head()

,1012a_mean,1012a_sum
0,5.304348,244.0
1,11.608696,534.0
2,2.304348,106.0
3,36.673913,1687.0
4,0.564103,22.0


#### 3.3.1.4. bus_bts를 활용한 변수

* 카테고리별 승객 수의 합과 비율

In [37]:
# 카테고리별 승객 수의 합
def category_people():
    # df와 병합을 위해 bus_route_id의 타입을 str로 변환
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    # bus_route_id와 user_category별 승객 수의 합
    f = bts.groupby(
        ['bus_route_id','user_category']
    )['user_count'].agg([('승객수', 'sum')]).reset_index()

    # bus_route_id별 user_category 변수의 승객수의 피벗 테이블 형태로 변환
    g = pd.pivot_table(f, values='승객수', index='bus_route_id', 
                       columns='user_category', fill_value=0).reset_index()
    g.columns = ['bus_route_id', 'adult','kids','teen','elder','jang','jang2','ugong','ugong2']
    g = g[['bus_route_id', 'adult','kids','teen','elder']]

    # 기존의 데이터프레임에 새로운 변수를 병합
    merge_data = pd.merge(df, g, how='left', on='bus_route_id')
                 
    return merge_data

# bus_route_id의 타입을 str로 통일시켜준 뒤, 함수를 실행하여 변수 생성
df['bus_route_id'] = df['bus_route_id'].astype(str)
df = category_people()

In [38]:
df[['adult','kids','teen','elder']].head()

,adult,kids,teen,elder
0,8158.0,72.0,38.0,0.0
1,8158.0,72.0,38.0,0.0
2,8158.0,72.0,38.0,0.0
3,8158.0,72.0,38.0,0.0
4,8158.0,72.0,38.0,0.0


In [39]:
# 카테고리별 승객 비율
def category_people_ratio() :

    # bus_route_id별 user_category의 전체 합
    f_all = bts.groupby('bus_route_id'
                       )['user_count'].agg([('전체', 'sum')]).reset_index()

   # bus_route_id와 user_category별 승객 수의 합
    f_sum = bts.groupby(['bus_route_id','user_category']
                       )['user_count'].agg([('승객수', 'sum')]).reset_index()
    
    # 승객 수의 합을 전체 합으로 나눠줘 비율을 구함
    data = pd.merge(f_all, f_sum, on='bus_route_id')
    data['비율'] = data['승객수'] / data['전체']

   # 피벗 테이블을 통해 bus_route_id별 user_category변수의 승객 비율 형태로 변환
    data = pd.pivot_table(data, values='비율', index='bus_route_id', 
                          columns='user_category',fill_value=0).reset_index()
    data.columns = ['bus_route_id', 'adult_prop','kids_prop','teen_prop',
                    'elder_prop','jang_prop','jang2_prop','ugong_prop','ugong2_prop']
    f = data[['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop']]

    # 기존의 데이터프레임에 새로운 변수를 병합
    merge_data = pd.merge(df, f, how='left', on='bus_route_id')

    return merge_data

# 함수를 실행하여 변수 생성
df = category_people_ratio()

In [40]:
df[['adult_prop','kids_prop','teen_prop','elder_prop']].head()

,adult_prop,kids_prop,teen_prop,elder_prop
0,0.986696,0.008708,0.004596,0.0
1,0.986696,0.008708,0.004596,0.0
2,0.986696,0.008708,0.004596,0.0
3,0.986696,0.008708,0.004596,0.0
4,0.986696,0.008708,0.004596,0.0


In [41]:
# 연속형 변수의 결측치를 평균값으로 처리할 수 있도록 SimpleImputer를 사용
from sklearn.impute import SimpleImputer  

# SimpleImputer를 사용할 변수 추출
na_col = ['adult', 'kids', 'teen', 'elder', 'adult_prop', 'kids_prop', 'teen_prop',
          'elder_prop']
# 평균값으로 결측치 처리를 위해 strategy옵션을 mean으로 설정 
imputer = SimpleImputer(strategy="mean")
# 각 변수에 대해 평균값을 구하고 결측치 대치를 위해 fit_transform을 사용
df[na_col] = pd.DataFrame(imputer.fit_transform(df[na_col]), columns = na_col)

#### 3.3.1.5 좌표를 활용한 변수

In [45]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) # 제주시의 위도 경도
coords_seoquiposi = (33.259429, 126.558217) # 서귀포시의 위도 경도

# 제주시와의 거리
df['dis_jejusi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(df))]
# 서귀포시와의 거리
df['dis_seoquiposi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_seoquiposi).km for i in range(len(df))]

In [46]:
df[['dis_jejusi', 'dis_seoquiposi']].head()

,dis_jejusi,dis_seoquiposi
0,2.954920,26.256744
1,3.720275,26.403025
2,5.036124,25.893305
3,2.864166,27.997494
4,29.040353,13.574693


* 탑승하는 승객의 수가 많은 버스정류장과의 거리

In [ ]:
# 승차 인원 상위 10개의 버스정류장 좌표
coords_jejusicheong1 = (33.49892, 126.53035) # 제주시청(광양 방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) # 제주국제공항(구제주 방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) # 한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) # 제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) # 동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) # 제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) # 중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양 방면)의 위도 경도

# 승차 인원 상위 10개의 버스정류장 좌표와 각 정류장의 지구 표면에서 두 점 사이의 거리를 구함
df['dis_jejusicheong1'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(df))]
df['dis_jejuairport'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(df))]
df['dis_hallahosp'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(df))]
df['dis_rotary'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_rotary).km for i in range(len(df))]
df['dis_jejucenterhigh'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(df))]
df['dis_jejumarket'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(df))]
df['dis_jejusclass'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(df))]
df['dis_centerroad'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_centerroad).km for i in range(len(df))]
df['dis_fiveway'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_fiveway).km for i in range(len(df))]
df['dis_law'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_law).km for i in range(len(df))]

In [ ]:
df[['dis_jejusicheong1', 'dis_jejuairport', 'dis_hallahosp', 'dis_rotary',
    'dis_jejucenterhigh', 'dis_jejumarket', 'dis_jejusclass', 'dis_centerroad',
    'dis_fiveway', 'dis_law']].head()

### 3.3.2. 외부 데이터를 통한 변수 생성

#### 3.3.2.1. 날씨를 활용한 변수

* 버스정류장과 각 측정소의 거리

In [ ]:
# 측정소별 위도, 경도
jeju = (33.51411, 126.52969) # 제주 측정소 근처
gosan = (33.29382, 126.16283) # 고산 측정소 근처
seongsan = (33.38677, 126.8802) # 성산 측정소 근처
po = (33.24616, 126.5653) # 서귀포 측정소 근처

# 버스정류장과 제주 측정소와의 거리
t1 = [geopy.distance.geodesic((i,j), jeju).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_jeju'] = t1

# 버스정류장과 고산 측정소와의 거리
t2 = [geopy.distance.geodesic((i,j), gosan).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_gosan']=t2

# 버스정류장과 성산 측정소와의 거리
t3 = [geopy.distance.geodesic((i,j), seongsan).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_seongsan']=t3

# 버스정류장과 서귀포 측정소와의 거리
t4 = [geopy.distance.geodesic((i,j), po).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_po']=t4

In [ ]:
df[['dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po']].head()

* 버스정류장과 가장 가까운 관측소

In [ ]:
total = pd.DataFrame(list(zip(t1,t2,t3,t4)),
                     columns=['jeju', 'gosan', 'seongsan', 'po'])
df['dist_name'] = total.apply(lambda x: x.idxmin(), axis=1)

In [ ]:
df[['dist_name']].head()

* 지점별 기상정보 변수

In [ ]:
# 변수명 변경 
# ['지점', '일시', '기온(°C)', '강수량(mm)']
rain.columns = ['dist_name', 'date', 'temperature', 'rainfall']

# fillna 함수를 활용해 결측치를 0으로 치환
rain = rain.fillna(0)

# rain 데이터의 date 변수를 datetime 형태로 변환
rain['date'] = pd.to_datetime(rain['date'])

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, rain, how='left', on=['dist_name', 'date'])

In [ ]:
df[['temperature', 'rainfall']].head()

* rainy_day

In [ ]:
def is_rainy(x):
    if x == 0:
        return 0 # 비 안오는 날
    else:
        return 1 # 비 오는 날

df['rainy_day'] = df['rainfall'].apply(is_rainy)

In [ ]:
df[['rainy_day']].head()

#### 3.3.2.2. jeju_financial_life_data를 활용한 변수

In [ ]:
# df 주소 데이터(df_location.csv) 로드
df_loc = pd.read_csv("df_location.csv", encoding='cp949')[['location','dong', 'si']]
# jeju_life 주소 데이터(life_location.csv) 로드
life_loc = pd.read_csv("life_location.csv", encoding='cp949'
                      )[['location','dong', 'si']]

# df의 위도, 경도 변수를 문자열로 취급하여 location 변수 생성
df['location'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)
# jeju_life의 위도, 경도 변수를 문자열로 취급하여 location 변수 생성
jeju_life['location'] = jeju_life['y_axis'].astype(str).str[:10] + ',' + \
                        jeju_life['x_axis'].astype(str).str[:10]

# location 변수를 기준으로 df와 df_loc 병합
df = pd.merge(df, df_loc, how='left', on='location')
# location 변수를 기준으로 jeju_life와 life_loc 병합
jeju_life = pd.merge(jeju_life, life_loc, how='left', on='location')

In [ ]:
df[['location', 'dong', 'si']].head()

* 각 동(읍, 면별) 직업, 소득, 소비, 부동산 관련 변수의 평균, 합계, 비율

In [ ]:
col = ['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self', 
       'avg_income', 'med_income', 'avg_spend', 'vehicle_own_rat']

# groupby 함수를 사용해 dong을 기준으로 위에서 선택한 변수들의 평균을 구함
data1 = jeju_life.groupby(['dong'])[col].mean().reset_index()
data1.columns=['dong','mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 
               'mean_job_profession', 'mean_job_self', 'mean_vehicle_own_rat',
               'mean_avg_income', 'mean_med_income', 'mean_avg_spend']

# groupby 함수를 사용해 dong을 기준으로 위에서 선택한 변수들의 합계를 구함 
data2 = jeju_life.groupby(['dong'])[col].sum().reset_index()
data2.columns=['dong','sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 
               'sum_job_profession', 'sum_job_self', 'sum_vehicle_own_rat', 
               'sum_avg_income', 'sum_med_income', 'sum_avg_spend']

# groupby 함수를 사용해 dong을 기준으로 위에서 선택한 변수들의 비율을 구함
data3 = (jeju_life.groupby(['dong'])[col].sum() \
             / jeju_life.groupby(['dong'])[col].sum().sum()).reset_index()
data3.columns = ['dong','rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 
                 'rate_job_profession', 'rate_job_self', 'rate_vehicle_own_rat', 
                 'rate_avg_income', 'rate_med_income', 'rate_avg_spend']

# dong을 기준으로 기존의 데이터프레임에 새로운 변수를 병합
merge_data1 = pd.merge(data1, data2, how='left', on='dong')
merge_data2 = pd.merge(merge_data1, data3, how='left', on='dong')
df = pd.merge(df, merge_data2, how='left', on='dong')

In [ ]:
df[['mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 
    'mean_job_profession', 'mean_job_self', 'mean_vehicle_own_rat', 
    'mean_avg_income', 'mean_med_income', 'mean_avg_spend']].head()

In [ ]:
df[['sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 
    'sum_job_profession', 'sum_job_self', 'sum_vehicle_own_rat', 
    'sum_avg_income', 'sum_med_income', 'sum_avg_spend']].head()

In [ ]:
df[['rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 
    'rate_job_profession', 'rate_job_self', 'rate_vehicle_own_rat', 
    'rate_avg_income', 'rate_med_income', 'rate_avg_spend']].head()

In [ ]:
# 새로 생성한 27개의 변수 정보
df[merge_data2.columns[1:]].info() # 새로 생성한 27개 변수 각각 201개의 NA값이 존재

In [ ]:
# 결측치를 평균값으로 처리할 수 있도록 SimpleImputer를 사용
imputer = SimpleImputer(strategy="mean")

# 결측치가 존재하는 변수 목록
na_col = list(merge_data2.columns[1:])

# 각 변수에 대해 평균값을 구하고 결측치 대치를 위해 fit_transform을 사용
df[na_col] = pd.DataFrame(imputer.fit_transform(df[na_col]), columns = na_col)

In [ ]:
# NA값 확인
df[na_col].isnull().sum()

### 3.3.3. 추가 변수 생성

<font color='blue'>3.3.3. '추가 변수 생성'에서 생성하는 변수는 책에는 나와있지 않으므로 주피터 파일을 참고해주세요.

* 승하차 시간대 통합 변수(t~t+3)

In [ ]:
# 오전시간대에 3시간동안 승차한 인원수의 합계
df['69a'] = df['6~7_ride'] + df['7~8_ride'] + df['8~9_ride']
df['912a']= df['9~10_ride'] + df['10~11_ride'] + df['11~12_ride']

# 오전시간대에 3시간동안 하차한 인원수의 합계
df['69b'] = df['6~7_takeoff'] + df['7~8_takeoff'] + df['8~9_takeoff']
df['912b'] = df['9~10_takeoff'] + df['10~11_takeoff'] + df['11~12_takeoff']

In [ ]:
df[['69a', '912a', '69b', '912b']].head()

* 제주도 기상정보 변수

In [ ]:
# 변수명 변경
# ['날짜', '현재일기', '현재기온', '체감온도', '일강수']
weather.columns = ['date','description','temp','feels_like','rain']
weather['rain'] = weather['rain'].replace(' ', '0')
weather['rain'] = weather['rain'].astype(float)
weather['date'] = pd.to_datetime(weather['date'])
df = pd.merge(df, weather, how='left', on='date')

In [ ]:
df[['description', 'temp', 'feels_like', 'rain']].head()

* id별 퇴근 시간 총/평균 승객수

In [ ]:
def id_statistic(ID, col1, col2) :
    
    train_data = df.query('cue==0').reset_index()    

    # mean, sum
    rs_mean = train_data.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]
                                                        ).reset_index()
    rs_sum = train_data.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]
                                                       ).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on=ID)

    # merge
    merge_data = pd.merge(df, rs_mean_sum, how='left', on=ID)

    # 연속형 변수의 결측치를 평균값으로 대체
    merge_data[col1] = merge_data[col1].fillna(rs_mean[col1].mean())
    merge_data[col2] = merge_data[col2].fillna(rs_sum[col2].mean())
    
    return merge_data


df = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')
df = id_statistic('station_code', '1820_s_mean', '1820_s_sum')
df = id_statistic('weekday', '1820_w_mean', '1820_w_sum')
df = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')

In [ ]:
df[['1820_r_mean', '1820_r_sum', '1820_s_mean', '1820_s_sum',
   '1820_w_mean', '1820_w_sum', '1820_rs_mean', '1820_rs_sum']].head()

* 라벨 인코딩

In [ ]:
# sklearn라이브러리에서 라벨 인코딩 객체 불러와서 저장하기
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

# 라벨 인코딩 할 변수 선택
df_encode = df[['description', 'bus_route_id','station_code']]
# fit_transform을 통해 라벨 인코딩 
df_encoded = df_encode.apply(labelencoder.fit_transform)

df['description'] = df_encoded['description']
df['bus_route_id2'] = df_encoded['bus_route_id']
df['station_code2'] = df_encoded['station_code']

In [ ]:
df[['description', 'bus_route_id2', 'station_code2']].head()

### 3.3.4. 라벨 인코딩과 원핫 인코딩 변수

#### 3.3.4.1. 라벨 인코딩 변수

* in_out

In [65]:
df['in_out'] = df['in_out'].map({'시내':0, '시외':1})

In [66]:
df[['in_out']].head()

,in_out
0,1
1,1
2,1
3,0
4,0


* 주말 및 주중

In [67]:
def is_weekend(x):
    if x == 5: # 토요일
        return 1
    elif x == 6: # 일요일
        return 1
    else:
        return 0
# 'weekday’변수에 is_weekend함수를 적용하여 'weekend' 변수 생성      
df['weekend'] = df['weekday'].apply(is_weekend)

In [68]:
df[['weekend']].head()

,weekend
0,1
1,1
2,1
3,1
4,1


* 시, 동

In [69]:
labelencoder = LabelEncoder()

# 라벨 인코딩 할 변수 선택
df_encode = df[['si', 'dong']]
# fit_transform을 통해 라벨 인코딩 
df_encoded = df_encode.apply(labelencoder.fit_transform)

df['si']=df_encoded['si']
df['dong']=df_encoded['dong']

In [70]:
df[['si', 'dong']].head()

,si,dong
0,1,36
1,1,36
2,1,5
3,1,47
4,0,26


#### 3.3.4.2. 원핫 인코딩 변수

In [71]:
df = pd.get_dummies(df, columns=['weekday', 'dist_name'])

In [72]:
df[['weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
    'weekday_4', 'weekday_5', 'weekday_6']].head()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [73]:
df[['dist_name_gosan','dist_name_jeju','dist_name_po','dist_name_seongsan']].head()

,dist_name_gosan,dist_name_jeju,dist_name_po,dist_name_seongsan
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,0,1,0


In [74]:
df.shape

(643593, 131)

## 3.4. 모델 구축과 검증

In [75]:
# 문자 형태의 변수 추출
object_column = list(df.columns[df.dtypes == 'object'])
# 문자형 변수 제거
df = df.drop(object_column, axis='columns')

# datetime 형태의 변수 추출
datetime_column = list(df.columns[df.dtypes == 'datetime64[ns]'])
# datetime형 변수 제거
df = df.drop(datetime_column, axis='columns')

In [76]:
print(object_column + datetime_column)

['bus_route_id', 'station_name', 'route_station', 'location', 'date']


In [77]:
df.shape

(643593, 126)

In [78]:
# 학습 데이터
X_train = df.query("cue==0").drop(['cue', '18~20_ride'], axis='columns')
# 테스트 데이터
X_test = df.query("cue==1").drop(['cue', '18~20_ride'], axis='columns')
# 타깃 변수
y_train = df.query("cue==0")['18~20_ride']

In [79]:
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)

(415423, 124)

(228170, 124)

(415423,)

### 3.4.1. 머신러닝 모델

#### 3.4.1.1. 배깅 방식 앙상블 모델

* 랜덤포레스트

In [80]:
%%time

from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=1217)
# 모델 학습
rf_model.fit(X_train, y_train)
# 예측값 생성
y_test = rf_model.predict(X_test)

Wall time: 21min 35s


#### 3.4.1.2. 부스팅 방식 앙상블 모델

* XGBoost

In [81]:
%%time

from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state=1217)
# 모델 학습
xgb_model.fit(X_train, y_train)
# 예측값 생성
y_test = xgb_model.predict(X_test)

Wall time: 21.2 s


* LightGBM

In [82]:
%%time

import lightgbm as lgb

lgbm = lgb.LGBMRegressor(random_state=1217)
# 모델 학습
lgbm.fit(X_train, y_train)
# 예측값 생성
y_test = lgbm.predict(X_test)

Wall time: 2.61 s


### 3.4.2. 모델 검증

#### 3.4.2.1. 교차검증

In [83]:
%%time

from sklearn.model_selection import cross_val_score

rf_model = RandomForestRegressor(random_state=1217)

# cv=5인 교차 검증
scores = cross_val_score(rf_model, X_train, y_train, cv=5, n_jobs=-1,
                         scoring = 'neg_mean_squared_error')

# 성능 확인
print('cross_val_score \n{}'.format(np.sqrt(-scores)))
print('cross_val_score.mean \n{:.3f}'.format(np.sqrt(-scores.mean())))

cross_val_score 
[2.38605365 2.47575838 2.53629148 2.36468601 2.2320487 ]
cross_val_score.mean 
2.401
Wall time: 19min 49s


### 3.4.3. 변수 선택

#### 3.4.3.1. 최종 데이터 구축

In [84]:
input_var1=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride',
            '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
            '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
            'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
            'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejusicheong1',
            'dis_jejuairport', 'dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
            'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway',
            'dis_law', 'weekend', 'holiday', 'ride_sum', 'takeoff_sum',
            '1820_r_mean', '1820_s_mean', '1820_w_mean', '1820_rs_mean',
            'station_code2', 'bus_route_id2', 'rain', 'description', 'feels_like',
            '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum',
            '10~11_ride_sum', 'in_out_mean', 'weekdaymean', 'adult', 'kids', 'teen',
            'elder', 'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
            'mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 
            'mean_job_profession', 'mean_job_self', 'mean_vehicle_own_rat',
            'mean_avg_income', 'mean_med_income', 'mean_avg_spend',
            'rate_job_majorc', 'rate_job_smallc', 'rate_job_public',
            'rate_job_profession', 'rate_job_self', 'rate_vehicle_own_rat', 
            'rate_avg_income', 'rate_med_income', 'rate_avg_spend',
            'sum_job_majorc', 'sum_job_smallc', 'sum_job_public',
            'sum_job_profession', 'sum_job_self', 'sum_vehicle_own_rat',
            'sum_avg_income', 'sum_med_income', 'sum_avg_spend', '68a', '810a',
            '1012a', '68b', '810b', '1012b', '69a', '912a', '69b', '912b',
            'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', 'dist_name_gosan',
            'dist_name_jeju', 'dist_name_po', 'dist_name_seongsan', 'temperature',
            'rainfall', 'school', 'transfer', 'si', 'dong', 'rainy_day']

In [85]:
input_var2=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride',
            '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
            '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
            'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 
            'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejusicheong1',
            'dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
            'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway',
            'dis_law', 'weekend', 'holiday', 'ride_sum', 'takeoff_sum',
            '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'station_code2',
            'bus_route_id2', 'rain', 'description', 'feels_like', '1820_w_mean',
            'in_out_mean', 'weekdaymean', 'adult', 'kids', 'teen', 'elder',
            'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop', 'mean_job_majorc',
            'mean_job_smallc', 'mean_job_public', 'mean_job_profession',
            'mean_job_self', 'mean_vehicle_own_rat', '68a', '810a', '1012a', '68b',
            '810b', '1012b', 'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po',
            'dist_name_gosan','dist_name_jeju', 'dist_name_po', 'dist_name_seongsan',
            'temperature', 'rainfall']

In [86]:
input_var3=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride',
            '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
            '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
            'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
            'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejuairport', 'ride_sum',
            'takeoff_sum','1820_r_mean','1820_r_sum', '1820_s_mean', '1820_s_sum', 
            '1820_w_mean', '1820_w_sum', '1820_rs_mean', '1820_rs_sum',
            'bus_route_id2', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum',
            '9~10_ride_sum', '10~11_ride_sum', 'sum_avg_spend', '68a', '810a', 'si',
            'dong']

### 3.4.4. 하이퍼파라미터 튜닝

#### 3.4.4.1. 그리드 서치

In [87]:
%%time

from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=1217)

# 탐색할 하이퍼파라미터 값 지정
params = {'n_estimators' : [200, 300, 500],
          'max_features': [5, 6, 8],
          'min_samples_leaf' : [1, 3, 5]}

# GridSearchCV를 이용해 최적의 하이퍼파라미터 탐색
grid_cv = GridSearchCV(rf, param_grid = params, cv = 3, n_jobs=-1)
grid_cv.fit(X_train[input_var1], y_train)

# 최적의 하이퍼파라미터 값과 성능 출력
print('최적 하이퍼파라미터: ', grid_cv.best_params_)
print('최적 하이퍼파라미터의 성능(RMSE): {:.4f}'.format(
    np.sqrt(grid_cv.best_score_)))

최적 하이퍼파라미터:  {'max_features': 8, 'min_samples_leaf': 1, 'n_estimators': 500}
최적 하이퍼파라미터의 성능(RMSE): 0.8692
Wall time: 46min 54s


#### 3.4.4.2. 랜덤 서치

In [88]:
%%time

import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(random_state=1217)

# 탐색할 하이퍼파라미터 구간 지정
params = {'n_estimators': stats.randint(200, 500),
          'max_features': stats.randint(4, 8),
          'min_samples_leaf': stats.randint(1, 5)}

# RandomizedSearchCV를 이용해 최적의 하이퍼파라미터 탐색
rand_cv = RandomizedSearchCV(rf, param_distributions = params, n_iter=10, 
                             cv = 3, random_state = 1217, n_jobs=-1)
rand_cv.fit(X_train[input_var1], y_train)

# 최적의 하이퍼파라미터 값과 성능 출력
print('최적 하이퍼파라미터: ', rand_cv.best_params_)
print('최적 하이퍼파라미터의 성능(RMSE): {:.4f}'.format(np.sqrt(rand_cv.best_score_)))

최적 하이퍼파라미터:  {'max_features': 7, 'min_samples_leaf': 1, 'n_estimators': 225}
최적 하이퍼파라미터의 성능(RMSE): 0.8683
Wall time: 17min 8s


### 3.4.5. 최종 모델 구축

#### 3.4.5.2. 최종 모델 구축

In [89]:
# submission_sample 데이터 로드
sub = pd.read_csv('C:/dacon/ch03/submission_sample.csv')

In [90]:
sub_dir = "C:/dacon/ch03/submission/" 

# submission 폴더가 없다면 submission 폴더를 생성
if not os.path.isdir(sub_dir):
    os.mkdir(sub_dir)

# 작업 경로를 submission 폴더로 변경
os.chdir(sub_dir)

* 첫 번째 데이터로 만든 모델

In [91]:
%%time

# 첫 번째 모델 정의
model1 = lgb.LGBMRegressor(random_state=1217, num_iterations = 1000, learning_rate = 0.05,
                           boosting = 'dart', Metric = 'regression_l2', n_jobs=-1)
# 모델 학습
model1.fit(X_train[input_var1], y_train)
# 예측값 생성
pred1 = model1.predict(X_train[input_var1])
# 성능 확인
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train, pred1))

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
Wall time: 42 s


1.7232868632994225

In [92]:
# X_test 데이터에 대한 예측값 생성
y_pred1 = model1.predict(X_test[input_var1])
sub['18~20_ride'] = y_pred1
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model1_lgbm.csv', index = False)

In [93]:
%%time

# 두 번째 모델 정의
model2 = RandomForestRegressor(random_state=1217, max_features=8, 
                               min_samples_leaf=1, n_estimators=500, n_jobs=-1)
# 모델 학습
model2.fit(X_train[input_var1], y_train)
# 예측값 생성
pred2 = model2.predict(X_train[input_var1])
# 성능 확인
np.sqrt(mean_squared_error(y_train, pred2))

Wall time: 1min 18s


0.8198244805339306

In [94]:
# X_test 데이터에 대한 예측값 생성
y_pred2 = model2.predict(X_test[input_var1])
sub['18~20_ride'] = y_pred2
# 'submission' 폴더에 CSV 파일 저장
sub.to_csv('model2_rf.csv', index = False)

In [95]:
%%time

# 세 번째 모델 정의
model3 = RandomForestRegressor(random_state=1217, max_features=3, 
                               min_samples_leaf=2, n_estimators=500, n_jobs=-1)
# 모델 학습
model3.fit(X_train[input_var1], y_train)
# 예측값 생성
pred3 = model3.predict(X_train[input_var1])
# 성능 확인
np.sqrt(mean_squared_error(y_train, pred3))

Wall time: 38.2 s


1.5304048954108196

In [96]:
# X_test 데이터에 대한 예측값 생성
y_pred3 = model3.predict(X_test[input_var1])
sub['18~20_ride'] = y_pred3
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model3_rf.csv', index = False)

* 두 번째 데이터로 만든 모델

In [97]:
%%time

# 네 번째 모델 정의
model4 = RandomForestRegressor(random_state=1217, max_features=4, 
                               min_samples_leaf=2, n_estimators=500, n_jobs=-1)
# 모델 학습
model4.fit(X_train[input_var2], y_train)
# 예측값 생성
pred4 = model4.predict(X_train[input_var2])
# 성능 확인
np.sqrt(mean_squared_error(y_train, pred4))

Wall time: 42.9 s


1.441432134120466

In [98]:
# X_test 데이터에 대한 예측값 생성
y_pred4 = model4.predict(X_test[input_var2])
sub['18~20_ride'] = y_pred4
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model4_rf.csv', index = False)

* 세 번째 데이터로 만든 모델

In [99]:
%%time

# 다섯 번째 모델 정의
model5 = RandomForestRegressor(random_state=1217, max_features=7, 
                               min_samples_leaf=4, n_estimators=300, n_jobs=-1)
# 모델 학습
model5.fit(X_train[input_var3], y_train)
# 예측값 생성
pred5 = model5.predict(X_train[input_var3])
# 성능 확인
np.sqrt(mean_squared_error(y_train, pred5))

Wall time: 32 s


1.5477565412182086

In [100]:
# X_test 데이터에 대한 예측값 생성
y_pred5 = model5.predict(X_test[input_var3])
sub['18~20_ride'] = y_pred5
# 'submission' 폴더에 csv 파일 저장
sub.to_csv('model5_rf.csv', index = False)